In [44]:
import numpy as np
import pandas as pd
import sodapy as sp
import pandas_bokeh as pbk
from sodapy import Socrata
import matplotlib.pyplot as plt

In [45]:
client = Socrata("analisi.transparenciacatalunya.cat", None)
results = client.get("rmgc-ncpb", limit=16000)
# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)
df.head()

,any,zona,dat,via,pk,nommun,nomcom,nomdem,f_morts,f_ferits_greus,...,d_superficie,d_tipus_via,d_titularitat_via,d_tracat_altimetric,d_vent,grupdialab,hor,gruphor,tipacc,tipdia
0,2010,Zona urbana,2010-01-25T23:33:00.000,SE,999999,CANOVES I SAMALUS,Valles Oriental,Barcelona,0,1,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NA,NA,"Calma, vent molt suau",Feiners,2333,Nit,Col.lisió de vehicles en marxa,dill-dij
1,2010,Carretera,2010-10-31T01:00:00.000,N-240,999,LLEIDA,Segria,Lleida,0,1,...,Sec i net,Carretera convencional,Estatal,Pla,"Calma, vent molt suau",CapDeSetmana,1,Nit,Sortida de la calcada sense especificar,dg
2,2010,Carretera,2010-05-17T15:27:00.000,N-II,7087,FORNELLS DE LA SELVA,Girones,Girona,1,0,...,Sec i net,Carretera convencional,Estatal,Rampa o pendent,"Calma, vent molt suau",Feiners,1527,Tarda,Col.lisió de vehicles en marxa,dill-dij
3,2010,Zona urbana,2010-08-21T22:30:00.000,SE,999999,BARCELONA,Barcelones,Barcelona,0,2,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NA,NA,"Calma, vent molt suau",CapDeSetmana,223,Nit,Col.lisió de vehicles en marxa,dis
4,2010,Zona urbana,2010-05-07T17:45:00.000,SE,999999,BADALONA,Barcelones,Barcelona,0,1,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NA,NA,"Calma, vent molt suau",CapDeSetmana,1745,Tarda,Bolcada a la calcada,div


In [70]:
df_time = pd.to_datetime(df['hora'])
df_time = df_time.to_frame()
df_time.head()

,hora
index,
0,2021-11-19 23:33:00
1,2021-11-19 01:00:00
2,2021-11-19 15:27:00
3,2021-11-19 22:30:00
4,2021-11-19 17:45:00


In [69]:
morts = df['f_morts']
morts = morts.to_frame()
morts.head()

,f_morts
index,
0,0
1,0
2,1
3,0
4,0


In [81]:
hora_morts = pd.concat([df_time, morts], axis=1)
hora_morts.head()

,hora,f_morts
index,,
0,2021-11-19 23:33:00,0
1,2021-11-19 01:00:00,0
2,2021-11-19 15:27:00,1
3,2021-11-19 22:30:00,0
4,2021-11-19 17:45:00,0


In [82]:
hora_morts['f_morts'] = hora_morts['f_morts'].astype(int)
hora_morts['f_morts'].dtype

dtype('int64')

In [84]:
# data re-sampled based on an hour
# how many deaths were added by accident in an hour
hora_mortss = hora_morts.resample('30min', on='hora').f_morts.sum()
hora_mortss

hora
2021-11-19 00:00:00    19
2021-11-19 00:30:00    21
2021-11-19 01:00:00    20
2021-11-19 01:30:00    22
2021-11-19 02:00:00    12
2021-11-19 02:30:00    20
2021-11-19 03:00:00    16
2021-11-19 03:30:00    21
2021-11-19 04:00:00    24
2021-11-19 04:30:00    36
2021-11-19 05:00:00    27
2021-11-19 05:30:00    47
2021-11-19 06:00:00    37
2021-11-19 06:30:00    54
2021-11-19 07:00:00    50
2021-11-19 07:30:00    53
2021-11-19 08:00:00    47
2021-11-19 08:30:00    45
2021-11-19 09:00:00    50
2021-11-19 09:30:00    50
2021-11-19 10:00:00    49
2021-11-19 10:30:00    45
2021-11-19 11:00:00    53
2021-11-19 11:30:00    67
2021-11-19 12:00:00    59
2021-11-19 12:30:00    50
2021-11-19 13:00:00    64
2021-11-19 13:30:00    47
2021-11-19 14:00:00    63
2021-11-19 14:30:00    54
2021-11-19 15:00:00    44
2021-11-19 15:30:00    56
2021-11-19 16:00:00    69
2021-11-19 16:30:00    72
2021-11-19 17:00:00    70
2021-11-19 17:30:00    65
2021-11-19 18:00:00    74
2021-11-19 18:30:00    71
2021-11

In [47]:
# Tenim en la columnna 'dat' el dia i l'hora juntes, volem separar-les en dues noves columnes 'data' i 'hora'
dist_morts=df[["dat","f_morts"]]
#dist_morts.head()

# Canviem la T que separa dia i hora per identificar la separació dues vegades: amb ' ' i amb '-'
df['dat'] = df['dat'].str.replace('T',' -')
df[['data','hora']] = df["dat"].str.split(" ", 1, expand=True)
df['data']=df['data'].astype("datetime64")
df["hora"] = df["hora"].str.strip("-")
df.head()



,any,zona,dat,via,pk,nommun,nomcom,nomdem,f_morts,f_ferits_greus,...,d_titularitat_via,d_tracat_altimetric,d_vent,grupdialab,hor,gruphor,tipacc,tipdia,data,hora
0,2010,Zona urbana,2010-01-25 -23:33:00.000,SE,999999,CANOVES I SAMALUS,Valles Oriental,Barcelona,0,1,...,NA,NA,"Calma, vent molt suau",Feiners,2333,Nit,Col.lisió de vehicles en marxa,dill-dij,2010-01-25,23:33:00.000
1,2010,Carretera,2010-10-31 -01:00:00.000,N-240,999,LLEIDA,Segria,Lleida,0,1,...,Estatal,Pla,"Calma, vent molt suau",CapDeSetmana,1,Nit,Sortida de la calcada sense especificar,dg,2010-10-31,01:00:00.000
2,2010,Carretera,2010-05-17 -15:27:00.000,N-II,7087,FORNELLS DE LA SELVA,Girones,Girona,1,0,...,Estatal,Rampa o pendent,"Calma, vent molt suau",Feiners,1527,Tarda,Col.lisió de vehicles en marxa,dill-dij,2010-05-17,15:27:00.000
3,2010,Zona urbana,2010-08-21 -22:30:00.000,SE,999999,BARCELONA,Barcelones,Barcelona,0,2,...,NA,NA,"Calma, vent molt suau",CapDeSetmana,223,Nit,Col.lisió de vehicles en marxa,dis,2010-08-21,22:30:00.000
4,2010,Zona urbana,2010-05-07 -17:45:00.000,SE,999999,BADALONA,Barcelones,Barcelona,0,1,...,NA,NA,"Calma, vent molt suau",CapDeSetmana,1745,Tarda,Bolcada a la calcada,div,2010-05-07,17:45:00.000


In [53]:
# data re-sampled based on an hour
#df = df.resample('H', on='dat').price.sum()
#df.head()

hora=df[["hora"]]
type(df['hora'])

#hora.reset_index(level=0, inplace=True)
#hora = hora['hora'].value_counts()
#hora = hora.to_frame()
#hora.groupby('hora')['index'].sum()
#hora = hora.sort_index()
#hora.index = pd.to_datetime(hora.index)
#hora
#hora.index = [time_stamp.time() for time_stamp in hora.index]
#hora = hora.reset_index()
#hora
#hora

#hora.groupby(pd.Grouper(freq='10Min')).count().plot(kind='bar')
#hora['index'].groupby(pd.Grouper(freq='10Min'))

pandas.core.series.Series

In [14]:
df['hora']= pd.to_(df['hora'])
df.head()

AttributeError: module 'pandas' has no attribute 'to_int'